In [1]:
%matplotlib inline
from ipywidgets import interact, BoundedFloatText, FloatSlider, Layout, SliderStyle
from DMS_PID import Damped_Mass_Spring_Model_PID_Simulator
import numpy as np
import matplotlib.pyplot as plt

class Visualizer:
    def __init__(self):
        self.m_input = BoundedFloatText(value=1.0, min=0, max=10.0, step=0.1, description='質量:', disabled=False)
        self.c_input = BoundedFloatText(value=0.5, min=0, max=10.0, step=0.1, description='ダンパ:', disabled=False)
        self.k_input = BoundedFloatText(value=1.0, min=0, max=10.0, step=0.1, description='ばね:', disabled=False)
        self.m = 1.0
        self.c = 0.5
        self.k = 1.0
        
        # 各ウィジェットに対してset_model_paramをイベントハンドラとして登録
        self.m_input.observe(self.set_model_param, names='value')
        self.c_input.observe(self.set_model_param, names='value')
        self.k_input.observe(self.set_model_param, names='value')
        
        # ウィジェットを表示
        display(self.m_input, self.c_input, self.k_input)
        
        # スライダーを作成
        interact(self.on_slide, kp=FloatSlider(min=0.0, max=20.0, step=0.001, value=10.0, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue')), 
                 ki=FloatSlider(min=0.0, max=20.0, step=0.001, value=2.16, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue')), 
                 kd=FloatSlider(min=0.0, max=20.0, step=0.001, value=4.13, layout=Layout(width='100%'), style=SliderStyle(handle_color='blue')))

    # 入力変更時に呼ばれる関数。値を更新して出力
    def set_model_param(self, change):
        # 各入力ボックスの値を対応する属性に更新
        self.m = self.m_input.value
        self.c = self.c_input.value
        self.k = self.k_input.value
        print(f"Updated parameters: m={self.m}, c={self.c}, k={self.k}")
    
    # シミュレーション実行メソッド
    def simulate(self, param):
        m, c, k, kp, ki, kd = param
        dms = Damped_Mass_Spring_Model_PID_Simulator(m, k, c, kp, ki, kd)
    
        maxT = 30
        dt = 0.01
        dms.set_aim([0, 0], [10, 0], [10])
        dms.ctrl.set_param(kp, ki, kd)
        dms.ctrl.reset()
        self.is_attained = dms.execute_until_stationary(maxT=maxT, dt=dt, threshold=0.1)
        self.plot_result(dms)

    def plot_result(self, dms):
        plt.figure(figsize=(12, 4))
        x_array = np.array([x.full().flatten() for x in dms.history_x])
        u_array = np.array([u.full().flatten() for u in dms.history_u])[:, 0]
        u_array = np.append(u_array, None)

        plt.subplot(1, 2, 1)
        title_str = 'Damped-Mass-Spring Model PID Simulation: '
        if self.is_attained:
            title_str += 'Attained in %.3f sec\n' % (dms.time[-1])
        else:
            title_str += 'not attained\n'
        title_str += '(m, k, c) = (%.1f, %.1f, %.1f); ' % (dms.sys.model.m, dms.sys.model.k, dms.sys.model.c)
        title_str += '(kp, ki, kd) = (%.3f, %.3f, %.3f)' % (dms.ctrl.Kp, dms.ctrl.Ki, dms.ctrl.Kd)
        plt.title(title_str)
        
        plt.plot(dms.time, x_array[:, 0], label='Position')
        plt.plot(dms.time, x_array[:, 1], label='Velocity')
        plt.plot(dms.time, u_array, label='Control Input')
        plt.xlabel('Time (s)')
        plt.ylabel('State')
        plt.legend()
        plt.grid(True)

        plt.subplot(1, 2, 2)
        charpol = np.poly1d([dms.sys.model.m, dms.sys.model.c + dms.ctrl.Kd, dms.sys.model.k + dms.ctrl.Kp, dms.ctrl.Ki])
        min_rt = np.inf
        max_rt = -np.inf
        hz = 0.0
        wav_amp = 0.0
        for z in charpol.roots:
            if np.abs(z.imag) >= 1.0e-8:
                hz = np.abs(z.imag / (2*np.pi))
                wav_amp = z.real
            min_rt = min(min_rt, z.real)
            max_rt = max(max_rt, z.real)
            
        rt_dist = max_rt - min_rt
        if rt_dist == 0:
            rt_dist = 1.0
        la_min = min_rt - rt_dist
        la_max = max_rt + rt_dist
        la = np.linspace(la_min, la_max, 100)
        charval = list(map(charpol, la))

        chartitle = '$%.1f\lambda^3 + %.1f\lambda^2 + %.1f\lambda + %.1f$: ' % tuple(charpol.c)
        if max_rt <= 0:
            chartitle += 'Converges (%.3f)\n' % max_rt
        else:
            chartitle += 'Diverges (%.3f)\n' % max_rt
        if hz == 0.0:
            chartitle += 'no vibration'
        else:
            chartitle += 'vibration: %.3f Hz (%.3f) ' % (hz, wav_amp)

        plt.title(chartitle)
        plt.plot(la, charval)
        plt.xlabel('Lambda')
        plt.ylabel('Char pol value')
        plt.grid(True)

    # スライダー変更時に呼ばれる関数
    def on_slide(self, kp, ki, kd):
        self.simulate((self.m, self.c, self.k, kp, ki, kd))

vis = Visualizer()


BoundedFloatText(value=1.0, description='質量:', max=10.0, step=0.1)

BoundedFloatText(value=0.5, description='ダンパ:', max=10.0, step=0.1)

BoundedFloatText(value=1.0, description='ばね:', max=10.0, step=0.1)

interactive(children=(FloatSlider(value=10.0, description='kp', layout=Layout(width='100%'), max=20.0, step=0.…